# TIF360 Project

# Data Pre-processing

Main source: https://www.kaggle.com/code/rmonge/predicting-molecule-properties-based-on-its-smiles/notebook

### Import packages

UPDATED ON 15/05-2023  11:20

In [1]:
import os
import numpy as np
import pandas as pd



In [16]:
# Input is list of strings on the form ['a','b','c']
# where the string are tokens
# Dictionary is the dictionary containign all possible tokens
# and an index for them
# MaxTokenLength is the max amount of tokens any input creates
def create_Onehot_Matrix(input, dictionary,maxTokenLength):
    # Create a matrix of zeros
    onehot_Matrix = np.zeros((len(dictionary),maxTokenLength))
    
    # Change value in right place to one
    keyCount = 0
    for key in input:
        onehot_Matrix[dictionary[key],keyCount] = 1
        keyCount+=1

    # Return it
    return(onehot_Matrix)

In [26]:

# Input is list of strings on the form ['a','b','c']
# where the string are tokens
# Dictionary is the dictionary containign all possible tokens
# and an index for them
# MaxTokenLength is the max amount of tokens any input creates
def create_encoded_vector(input, dictionary,maxTokenLength):
    # Create a matrix of zeros
    encoded_vector = np.zeros((1,maxTokenLength))
    print(encoded_vector)
    encoded_vector[0] = 10
    print(encoded_vector.shape)
    
    
    # Change value in right place to one
    keyCount = 0
    for key in input:
        encoded_vector[keyCount] = dictionary[0,key]
        keyCount+=1
    
    return encoded_vector

In [20]:
testDict = {}
testDict['a'] = 1
testDict['b'] = 2
testDict['c'] = 3

In [27]:
create_encoded_vector(['b','c'],testDict,5)

[[0. 0. 0. 0. 0.]]
(1, 5)


KeyError: (0, 'b')

In [3]:


df = pd.read_csv("../data/smiles_and_targets.csv")
print(np.shape(df))


(132820, 21)


In [4]:

properties_names = ['A', 'B', 'C', 'mu', 'alfa', 'homo', 'lumo', 'gap', 'R²', 'zpve', 'U0', 'U', 'H', 'G', 'Cv']

x_smiles = df.smiles.values
y = df.loc[:, properties_names].values  # shape = (n_samples, n_properties)


### Transformer Network

#### Model for all targets at once

Transformer

In [5]:
import atomInSmiles


testSmile = x_smiles[8]
print(testSmile)
tokens = atomInSmiles.encode(testSmile)
tokens = tokens.split()
print(tokens)
print(x_smiles.shape)



CC(C)C#N
['[CH3;!R;C]', '[CH;!R;CCC]', '(', '[CH3;!R;C]', ')', '[C;!R;CN]', '#', '[N;!R;C]']
(132820,)


In [6]:
# tokenize all smiles
import atomInSmiles

tokenList = []
for smile in x_smiles:
    tokenList.append(atomInSmiles.encode(smile).split())
#tokenList is target

# find longest word (max tokens from one smile+1)
maxTokenLength = 0
for token in tokenList:
    if len(token)>maxTokenLength:
        maxTokenLength = len(token)
print('Longest word (max amount of tokens):', maxTokenLength)

Longest word (max amount of tokens): 22


In [11]:

# Give each token a index in a dictionary
tokenDict = {}
count = 1

dictList = []
for itokens in tokenList:
    for itoke in itokens:
        #print(itoke)
        if tokenDict.get(itoke) == None:
            tokenDict[itoke] = count
        
            # current = [itoke, count]
            # dictList.append(current)
            count+=1

In [12]:

# Create one hot encoded matrix containing all the data made into encoded matrices
inputData = []
for token in tokenList:
    inputData.append(create_Onehot_Matrix(token,tokenDict,maxTokenLength))
inputData = np.asarray(inputData)

IndexError: index 149 is out of bounds for axis 0 with size 149

In [9]:
print(inputData[0])
print(inputData.shape)
print(inputData.type)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(132820, 149, 22)


In [10]:
import torch

class PositionalEncoding(torch.nn.Module):
    
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = torch.nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-np.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):

        x = x + self.pe[:x.size(0)]
        return self.dropout(x)


class TransformerLayer(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        torch.manual_seed(12345)
        self.Attention = torch.nn.MultiheadAttention(data_features,num_heads=3,dropout=0.15)
        self.Norm1 = torch.nn.LayerNorm(data_features)
        self.Dense1 = torch.nn.Linear(hidden_channels)
        self.Norm2 = torch.nn.LayerNorm(hidden_channels)
        self.Dense2 = torch.nn.Linear(hidden_channels)
        

    def forward(self, x):
        addNormX = x
        x = self.Attention(x)
        x = self.Norm1(x + addNormX)
        addNormX = x
        x = self.Dense1(x)
        x = self.Dense2(x)
        x = self.Norm2(x + addNormX)

       
 
        return x
    
class TransformerNetwork(torch.nn.Module):
    def __init__(self,hidden_channels,output_dim):
        super().__init__()
        torch.manual_seed(12345)
        self.EmbeddingLayer = torch.nn.Embedding(num_embeddings=148,embedding_dim = 600 , max_norm=True)
        self.PositionalEncoding = PositionalEncoding(d_model=1024)
        self.TransEnc1 = TransformerLayer(hidden_channels)
        
        self.Pooling = torch.nn.AvgPool1d()

        self.DenseOut1 = torch.nn.Linear(hidden_channels)
        self.DenseOut2 = torch.nn.Linear(output_dim)

    def forward(self,x):
        x = self.EmbeddingLayer(x)
        x = self.PositionalEncoding(x)
        x = self.TransEnc1(x)
        x = self.Pooling(x)
        x = self.DenseOut1(x)
        x = self.DenseOut2(x)


        return x



